## Домашнее задание к занятию "A/B-тесты"

### Описание задачи

![banner](https://storage.googleapis.com/kaggle-datasets-images/635/1204/126be74882028aac7241553cef0e27a7/dataset-original.jpg)

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) И относятся к одному или двум так называемым классам (вода, огонь и т. д.).
Профессор Оук является изобретателем Pokedex - портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание набора данных
Профессор Оук скопировал все содержимое в память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* `pid`: Numeric - ID покемона
* `HP`: Numeric - Очки здоровья
* `Attack`: Numeric - Сила обычной атаки
* `Defense`: Numeric - Сила обычной защиты
* `Sp. Atk`: Numeric - Сила специальной атаки
* `Sp. Def`: Numeric - Сила специальной защиты
* `Speed`: Numeric - Скорость движений
* `Legendary`: Boolean - «True», если покемон редкий
* `Class 1`: Categorical - Класс покемона
* `Class 2`: Categorical - Класс покемона

In [83]:
import warnings
# Отключение предупреждений (warnings)
warnings.filterwarnings("ignore")

import pandas as pd

from scipy.stats import ttest_ind
from scipy.stats import f_oneway, shapiro

pokemon = pd.read_csv('https://raw.githubusercontent.com/a-milenkin/datasets_for_t-tests/main/pokemon.csv', on_bad_lines='skip')  # Откроем датасет
pokemon.head()

# Обратите внимание, что у покемона может быть один или два класса.
# Если у покемона два класса, считается, что они имеют одинаковую значимость.

,pid,Name,Class 1,Class 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


### Задачи

<div class="alert alert-info">
<b>Задание № 1:</b>
    
Профессор Оук подозревает, что покемоны в классе `Grass` имеют более сильную обычную атаку, чем покемоны в классе `Rock`. Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
    
Примечание: если есть покемоны, которые относятся к обоим классам, просто выбросьте их;
    
Вы можете предположить, что распределение обычных атак является нормальным для всех классов покемонов.

</div>

In [84]:
pokemon.columns = pokemon.columns.str.replace(' ', '_')
pokemon.head()

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


In [85]:
pokemon_copy = pokemon.copy()
pokemon_copy.info()
pokemon_copy.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pid        800 non-null    int64 
 1   Name       799 non-null    object
 2   Class_1    800 non-null    object
 3   Class_2    414 non-null    object
 4   HP         800 non-null    int64 
 5   Attack     800 non-null    int64 
 6   Defense    800 non-null    int64 
 7   Sp._Atk    800 non-null    int64 
 8   Sp._Def    800 non-null    int64 
 9   Speed      800 non-null    int64 
 10  Legendary  800 non-null    bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 63.4+ KB


,pid,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed
count,800.0000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,400.5000,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500
std,231.0844,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474
min,1.0000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,200.7500,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000
50%,400.5000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000
75%,600.2500,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000
max,800.0000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000


In [23]:
#Находим покемонов сразу с двумя классами из условия
pokemon_2class = pokemon_copy.loc[
                                  ((pokemon_copy['Class_1'] == 'Rock' ) & (pokemon_copy['Class_2' ] == 'Grass')) |
                                  ((pokemon_copy['Class_1'] == 'Grass') & (pokemon_copy['Class_2' ] == 'Rock'))
                                  ]
pokemon_2class

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary
377,378,Lileep,Rock,Grass,66,41,77,61,87,23,False
378,379,Cradily,Rock,Grass,86,81,97,81,107,43,False


In [86]:
#Исключаем покемонов, имеющих два класса из условия
pokemon_copy_drop = pokemon_copy.drop(index=[377, 378]).reset_index()
pokemon_copy_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 798 entries, 0 to 797
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      798 non-null    int64 
 1   pid        798 non-null    int64 
 2   Name       797 non-null    object
 3   Class_1    798 non-null    object
 4   Class_2    412 non-null    object
 5   HP         798 non-null    int64 
 6   Attack     798 non-null    int64 
 7   Defense    798 non-null    int64 
 8   Sp._Atk    798 non-null    int64 
 9   Sp._Def    798 non-null    int64 
 10  Speed      798 non-null    int64 
 11  Legendary  798 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.5+ KB


In [87]:
#Формируем DF с покемонами, имеющими класс из условия
pokemon_copy_drop_grass = pokemon_copy_drop.loc[
                                                ((pokemon_copy_drop['Class_1'] == 'Grass' )) |
                                                ((pokemon_copy_drop['Class_2'] == 'Grass' ))
                                                ]
pokemon_copy_drop_rock = pokemon_copy_drop.loc[
                                                ((pokemon_copy_drop['Class_1'] == 'Rock' )) |
                                                ((pokemon_copy_drop['Class_2'] == 'Rock' ))
                                                ]

In [89]:
#Создаем функцию для проверки нормальности распределения
from scipy import stats

def check_normality(data):
    #H0 - Данные распределены нормально
    #H1 - Данные распределены не нормально
    #Уровень значимости alpha=0,05
    stat, p = stats.shapiro(data)
    if p < 0.05:
        print("Отклоняем нулевую гипотезу >> Данные распределены не нормально")
    else:
        print("Не отклоняем нулевую гипотезу >> Данные распределены нормально")

In [90]:
check_normality(pokemon_copy_drop_grass['Attack'])

Не отклоняем нулевую гипотезу >> Данные распределены нормально


In [91]:
check_normality(pokemon_copy_drop_rock['Attack'])

Не отклоняем нулевую гипотезу >> Данные распределены нормально


In [92]:
#Проверяем равенство вариаций
stat, p = stats.levene(pokemon_copy_drop_grass['Attack'], pokemon_copy_drop_rock['Attack'])

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print("Не отклоняем нулевую гипотезу >> Вариация в группах одинаковая")
else:
    print("Отклоняем нулевую гипотезу >> Вариация в группах различается")

Статистика = 8.93747, p = 0.00328
Отклоняем нулевую гипотезу >> Вариация в группах различается


In [93]:
#H0 - Средние выборок равны (различий между выборками нет)
#H1 - Средние выборок не равны (есть различия между выборками)
stat, p_val = stats.ttest_ind(a=pokemon_copy_drop_grass['Attack'], b=pokemon_copy_drop_rock['Attack'], equal_var = False)
print(f"Статистика = {stat:.5f}, p_val = {p_val:.5f}")

if p_val > 0.05:
    print("Не отклоняем нулевую гипотезу, средние выборок равны (различий между выборками нет)")
else:
    print("Отклоняем нулевую гипотезу, средние выборок не равны (есть различия между выборками)")

Статистика = -3.41694, p_val = 0.00096
Отклоняем нулевую гипотезу, средние выборок не равны (есть различия между выборками)


In [94]:
pokemon_grass_attack_mean = pokemon_copy_drop_grass['Attack'].mean()
pokemon_rock_attack_mean = pokemon_copy_drop_rock['Attack'].mean()

In [95]:
print(f'Средняя обычная атака покемонов с классом Grass - {pokemon_grass_attack_mean:.2f}')
print(f'Средняя обычная атака покемонов с классом Rock - {pokemon_rock_attack_mean:.2f}')

Средняя обычная атака покемонов с классом Grass - 73.73
Средняя обычная атака покемонов с классом Rock - 91.79


**ВЫВОД:**

Профессор Оук не прав.

Покемоны в классе `Grass` имеют **менее** сильную обычную атаку, чем покемоны в классе `Rock`. Разница статистически значимая.

<div class="alert alert-info">
<b>Задание № 2:</b>
    
Профессор Оук уже долго не может спать по ночам, ведь его волнует вопрос, а правда ли, что покемоны в классе `Water` в среднем быстрее, чем покемоны в классе `Normal`.
    
    
Проверьте, прав ли он, и убедите его в своём выводе статистически.
    
Примечание: если есть покемоны, которые относятся к обоим классам, выбросьте их;
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.
</div>

In [97]:
pokemon_2class_2 = pokemon_copy.loc[
                                    ((pokemon_copy['Class_1'] == 'Water' ) & (pokemon_copy['Class_2' ] == 'Normal')) |
                                    ((pokemon_copy['Class_1'] == 'Normal') & (pokemon_copy['Class_2' ] == 'Water'))
                                    ]
pokemon_2class_2

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary
445,446,Bibarel,Normal,Water,79,85,60,55,60,71,False


In [98]:
pokemon_copy_drop_2 = pokemon_copy.drop(index=445).reset_index()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      799 non-null    int64 
 1   pid        799 non-null    int64 
 2   Name       798 non-null    object
 3   Class_1    799 non-null    object
 4   Class_2    413 non-null    object
 5   HP         799 non-null    int64 
 6   Attack     799 non-null    int64 
 7   Defense    799 non-null    int64 
 8   Sp._Atk    799 non-null    int64 
 9   Sp._Def    799 non-null    int64 
 10  Speed      799 non-null    int64 
 11  Legendary  799 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.6+ KB


In [99]:
pokemon_copy_drop_2_water = pokemon_copy_drop_2.loc[
                                                    ((pokemon_copy_drop_2['Class_1'] == 'Water')) |
                                                    ((pokemon_copy_drop_2['Class_2'] == 'Water'))
                                                    ]
pokemon_copy_drop_2_normal = pokemon_copy_drop_2.loc[
                                                    ((pokemon_copy_drop_2['Class_1'] == 'Normal')) |
                                                    ((pokemon_copy_drop_2['Class_2'] == 'Normal'))
                                                    ]

In [100]:
check_normality(pokemon_copy_drop_2_water['Speed'])

Не отклоняем нулевую гипотезу >> Данные распределены нормально


In [101]:
check_normality(pokemon_copy_drop_2_normal['Speed'])

Не отклоняем нулевую гипотезу >> Данные распределены нормально


In [102]:
stat, p = stats.levene(pokemon_copy_drop_2_water['Speed'], pokemon_copy_drop_2_normal['Speed'])

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print("Не отклоняем нулевую гипотезу >> Вариация в группах одинаковая")
else:
    print("Отклоняем нулевую гипотезу >> Вариация в группах различается")

Статистика = 7.69447, p = 0.00601
Отклоняем нулевую гипотезу >> Вариация в группах различается


In [44]:
#H0 - Средние выборок равны (различий между выборками нет)
#H1 - Средние выборок не равны (есть различия между выборками)
stat, p_val = stats.ttest_ind(a=pokemon_copy_drop_2_water['Speed'], b=pokemon_copy_drop_2_normal['Speed'], equal_var = False)
print(f"Статистика = {stat:.5f}, p_val = {p_val:.5f}")

if p_val > 0.05:
    print("Не отклоняем нулевую гипотезу, средние выборок равны (различий между выборками нет)")
else:
    print("Отклоняем нулевую гипотезу, средние выборок не равны (есть различия между выборками)")

Статистика = -2.10119, p_val = 0.03695
Отклоняем нулевую гипотезу, средние выборок не равны (есть различия между выборками)


In [103]:
pokemon_grass_water_mean = pokemon_copy_drop_2_water['Speed'].mean()
pokemon_rock_normal_mean = pokemon_copy_drop_2_normal['Speed'].mean()

In [104]:
print(f'Средняя скорость покемонов с классом Water - {pokemon_grass_water_mean:.2f}')
print(f'Средняя скорость покемонов с классом Normal - {pokemon_rock_normal_mean:.2f}')

Средняя скорость покемонов с классом Water - 64.94
Средняя скорость покемонов с классом Normal - 72.26


**ВЫВОД:**

Не правда.

Покемоны в классе `Water` в среднем медленнее, чем покемоны в классе `Normal`. Разница статистически значимая.

<div class="alert alert-info">
<b>Задание № 3:</b>
    
Профессор Оук тот еще безумец. Он изобрёл сыворотку, способную ускорить покемона. Однако мы усомнились в эффективности его вакцины. Професоор дал эту сыворотку следующим покемонам: смотри массив `treathed_pokemon`. Проверьте, работает ли вообще его сыворотка, убедите всех в своём выводе статистически.
    
    
Вы можете предположить, что распределение скорости движения является нормальным для всех классов покемонов.

</div>

In [105]:
# Покемоны, которые принимали сыворотку увеличения скорости
treathed_pokemon = ['Mega Beedrill', 'Mega Alakazam',
                    'Deoxys Normal Forme', 'Mega Lopunny']

In [106]:
df_treathed_pokemon = pokemon_copy[pokemon_copy['Name'].isin(treathed_pokemon)]
df_treathed_pokemon

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary
19,20,Mega Beedrill,Bug,Poison,65,150,40,15,80,145,False
71,72,Mega Alakazam,Psychic,NaN,55,50,65,175,95,150,False
428,429,Deoxys Normal Forme,Psychic,NaN,50,150,50,150,50,150,True
476,477,Mega Lopunny,Normal,Fighting,65,136,94,54,96,135,False


In [107]:
df_other_pokemon = pokemon_copy.drop(index=[19, 71, 428, 476]).reset_index()

In [108]:
check_normality(df_treathed_pokemon['Speed'])
check_normality(df_other_pokemon['Speed'])

Не отклоняем нулевую гипотезу >> Данные распределены нормально
Отклоняем нулевую гипотезу >> Данные распределены не нормально


In [109]:
stat, p = stats.mannwhitneyu(df_treathed_pokemon['Speed'], df_other_pokemon['Speed'])

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

Статистика = 3164.50000, p = 0.00065
Отклоняем нулевую гипотезу, распределения, вероятно, различаются


**ВЫВОД:**

Тест показал наличие статистически значимой разницы в скорости - при текущих данных мы можем утверждать, что сыворотка все-таки изменяет скорость.

In [110]:
df_treathed_pokemon_speed = df_treathed_pokemon['Speed'].mean()
df_other_pokemon_speed = df_other_pokemon['Speed'].mean()

In [111]:
print(f'Средняя скорость покемонов, принявших сыворотку - {df_treathed_pokemon_speed:.2f}')
print(f'Средняя скорость покемонов, не принимавших сыворотку - {df_other_pokemon_speed:.2f}')

Средняя скорость покемонов, принявших сыворотку - 145.00
Средняя скорость покемонов, не принимавших сыворотку - 67.89


**ВЫВОД:**

По итогам статистического анализа выявлено, что есть статистически значимая разница между скоростью покемонов, принимавших сыворотку и не принимавших ее. Скорость покемонов, прнимавших сыворотку, выше, чем покемонов, которые ее не принимали.

<div class="alert alert-info">
<b>Задание № 4:</b>
    
Профессор Оук всегда любил истории про легендарных покемонов. Однако профессор не очень уверен, что они лучше остальных покемонов. Оук предложил разобраться в этом нам. Проверьте, действительно ли сумма характеристик `HP`,`Attack`,`Defense` у легендарных покемонов выше, чем у других покемонов?

А произведение этих же параметров?

Найдите ответы на эти вопросы и убедите всех в своём выводе статистически.
   

Вы можете предположить, что распределение сум и произведений этих параметров является нормальным для всех классов покемонов.

</div>

In [113]:
cols = ['HP', 'Attack', 'Defense']

In [114]:
pokemon_copy['sum_char'] = pokemon_copy[cols].sum(axis=1)

In [115]:
pokemon_copy['mult_char'] = pokemon_copy['HP']*pokemon_copy['Attack']*pokemon_copy['Defense']

In [116]:
pokemon_copy.head()

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary,sum_char,mult_char
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False,143,108045
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False,185,234360
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False,245,544480
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False,303,984000
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False,134,87204


In [117]:
legendary_pokemon = pokemon_copy.loc[pokemon_copy['Legendary'] == True]
other_pokemon = pokemon_copy.loc[pokemon_copy['Legendary'] == False]

In [118]:
check_normality(legendary_pokemon['sum_char'])
check_normality(other_pokemon['sum_char'])

Не отклоняем нулевую гипотезу >> Данные распределены нормально
Отклоняем нулевую гипотезу >> Данные распределены не нормально


In [119]:
check_normality(legendary_pokemon['mult_char'])
check_normality(other_pokemon['mult_char'])

Отклоняем нулевую гипотезу >> Данные распределены не нормально
Отклоняем нулевую гипотезу >> Данные распределены не нормально


In [120]:
stat_1, p_1 = stats.mannwhitneyu(legendary_pokemon['sum_char'], other_pokemon['sum_char'])

print(f"Статистика = {stat_1:.5f}, p = {p_1:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

Статистика = 42199.00000, p = 0.00000
Отклоняем нулевую гипотезу, распределения, вероятно, различаются


In [121]:
stat_2, p_2 = stats.mannwhitneyu(legendary_pokemon['mult_char'], other_pokemon['mult_char'])

print(f"Статистика = {stat_2:.5f}, p = {p_2:.5f}")

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, распределения, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, распределения, вероятно, различаются')

Статистика = 42033.50000, p = 0.00000
Отклоняем нулевую гипотезу, распределения, вероятно, различаются


In [126]:
legendary_pokemon_mean = legendary_pokemon['sum_char'].mean()
other_pokemon_mean = other_pokemon['sum_char'].mean()

In [127]:
print(f'Средняя сумма характеристик легендарных покемонов - {legendary_pokemon_mean:.2f}')
print(f'Средняя сумма характеристик остальных покемонов - {other_pokemon_mean:.2f}')

Средняя сумма характеристик легендарных покемонов - 309.08
Средняя сумма характеристик остальных покемонов - 214.41


**ВЫВОД:**

По итогам статистического анализа выявлено, что есть статистически значимая разница между средней суммой характеристик легендарных и остальных покемонов. Легендарные покемоны имеют лучший показатель исследуемых характеристик.

<div class="alert alert-info">
<b>Задание № 5:</b>
    
Профессор Оук частенько наблюдает за боями покемонов. После очередных таких боёв Оук выделил четыре класса `best_defence_class`, которые на его взгляд одинаковы по "силе обычной защиты" `Defense`.

Проверьте, действительно ли эти классы покемонов не отличаются по уровню защиты статистически значимо? Всё та же статистика вам в помощь!
   

Вы можете предположить, что распределение параметров защитных характеристик является нормальным для всех классов покемонов.

</div>

In [130]:
best_defence_class = ['Rock', 'Ground', 'Steel', 'Ice']
best_defence_class

['Rock', 'Ground', 'Steel', 'Ice']

In [131]:
pokemon.head()

,pid,Name,Class_1,Class_2,HP,Attack,Defense,Sp._Atk,Sp._Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


In [132]:
pokemon_class_rock = pokemon.loc[(pokemon_copy['Class_1'] == 'Rock' ) | (pokemon_copy['Class_2' ] == 'Rock')]
pokemon_class_ground = pokemon.loc[(pokemon_copy['Class_1'] == 'Ground' ) | (pokemon_copy['Class_2' ] == 'Ground')]
pokemon_class_steel = pokemon.loc[(pokemon_copy['Class_1'] == 'Steel' ) | (pokemon_copy['Class_2' ] == 'Steel')]
pokemon_class_ice = pokemon.loc[(pokemon_copy['Class_1'] == 'Ice' ) | (pokemon_copy['Class_2' ] == 'Ice')]

In [133]:
pokemon_class_rock_def = pokemon_class_rock['Defense']
pokemon_class_ground_def = pokemon_class_ground['Defense']
pokemon_class_steel_def = pokemon_class_steel['Defense']
pokemon_class_ice_def = pokemon_class_ice['Defense']

In [135]:
stat, p = stats.levene(pokemon_class_rock_def,pokemon_class_ground_def,pokemon_class_steel_def,pokemon_class_ice_def)

print(f"Статистика = {stat:.5f}, p = {p:.5f}")

if p <0.05:
    print("Отклоняем нулевую гипотезу >> Вариация в группах раличается")
else:
    print("Не отклоняем нулевую гипотезу >> Вариация в группах одинаковая")

Статистика = 0.48575, p = 0.69254
Не отклоняем нулевую гипотезу >> Вариация в группах одинаковая


In [136]:
fvalue, pvalue = stats.f_oneway(pokemon_class_rock_def,pokemon_class_ground_def,pokemon_class_steel_def,pokemon_class_ice_def)

print(f"Статистика = {fvalue:.5f}, p = {pvalue:.5f}")

if pvalue > 0.05:
    print('Не отклоняем нулевую гипотезу, средние, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, средние, вероятно, различаются')

Статистика = 10.82012, p = 0.00000
Отклоняем нулевую гипотезу, средние, вероятно, различаются


In [137]:
from scipy.stats import tukey_hsd

res = tukey_hsd(pokemon_class_rock_def,pokemon_class_ground_def,pokemon_class_steel_def,pokemon_class_ice_def)
print(res)

Tukey's HSD Pairwise Group Comparisons (95.0% Confidence Interval)
Comparison  Statistic  p-value  Lower CI  Upper CI
 (0 - 1)     19.385     0.024     1.868    36.901
 (0 - 2)     -9.526     0.563   -28.476     9.424
 (0 - 3)     30.428     0.001    10.045    50.811
 (1 - 0)    -19.385     0.024   -36.901    -1.868
 (1 - 2)    -28.911     0.000   -47.269   -10.552
 (1 - 3)     11.044     0.475    -8.790    30.877
 (2 - 0)      9.526     0.563    -9.424    28.476
 (2 - 1)     28.911     0.000    10.552    47.269
 (2 - 3)     39.954     0.000    18.843    61.065
 (3 - 0)    -30.428     0.001   -50.811   -10.045
 (3 - 1)    -11.044     0.475   -30.877     8.790
 (3 - 2)    -39.954     0.000   -61.065   -18.843



**ВЫВОД:**

Статистически значимая разница в характеристиках защиты имеется между покемонами следующих классов:
- Rock & Ground
- Rock & Ice
- Ground & Steel
- Steel & Ice

Вместе с тем существенных различий в характеристиках защиты не наблюдается между покемонами следующих классов:
- Rock & Steel
- Ground & Ice

Таким образом, покемоны, входящие в класс best_defence_class, по "силе обычной защиты" Defense имеют статистически значимую разницу.

# **Примечание:**

Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/). Мы не сможем проверить его или помочь, если вы пришлете:

*   файлы;
*   архивы;
*   скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в Discord.

**Как правильно задавать вопросы аспирантам, преподавателям и коллегам:**

Прежде чем задать вопрос, попробуйте найти ответ в интернете. Навык самостоятельного поиска информации — один из важнейших. Каждый практикующий специалист любого уровня делает это ежедневно.

Сформулируйте вопрос по алгоритму:

1.   Что я делаю?
2.   Какого результата я ожидаю?
3.   Как фактический результат отличается от ожидаемого?
4.   Что я уже попробовал сделать, чтобы исправить проблему?

По возможности прикрепите к вопросу скриншоты либо ссылки на код. Не выкладывайте все решение, оставляйте только проблемный и воспроизводимый участок кода.